In [1]:
## load data and funtions 
#########################################################################################
import sys
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict
# Add the workspace root to the Python path
workspace_root = os.path.abspath("..")
if workspace_root not in sys.path:
    sys.path.append(workspace_root)
from functions import preprocess, behaviorAnalysis
with open(os.path.join( workspace_root,'Data', 'bridged_channels_analysis.pkl'), "rb") as f:
    all_bridged_channels = pickle.load(f)

with open(os.path.join( workspace_root, 'Data', 'bad_channels_detected.pkl'), "rb") as f:
    all_bads = pickle.load(f)


def compute_ica(ids):

    import sys
    import os
    import numpy as np
    import mne
    import matplotlib.pyplot as plt
    import pickle
    from collections import defaultdict
    # Add the workspace root to the Python path
    workspace_root = os.path.abspath("..")
    if workspace_root not in sys.path:
        sys.path.append(workspace_root)
    from functions import preprocess, behaviorAnalysis
    with open(os.path.join( workspace_root,'Data', 'bridged_channels_analysis.pkl'), "rb") as f:
        all_bridged_channels = pickle.load(f)

    with open(os.path.join( workspace_root, 'Data', 'bad_channels_detected.pkl'), "rb") as f:
        all_bads = pickle.load(f)


    for id in ids:
        print(f"Processing subject {id}")
        try:
            bads_channel= all_bads[id]['channel_names']
            bad_trials= all_bads[id]['trial_numbers']
            bridged_channels= all_bridged_channels[5][id] 
            sub = preprocess(id)
            raw = sub.load_data()
            #############################################################################################
            # 1. remove noisy channels
            raw.info['bads'] = bads_channel
            # 2. Filter the data
            # raw.notch_filter([50,100], fir_design='firwin', skip_by_annotation='edge')
            raw.filter(l_freq=1, h_freq= None)
            # 3. segment the data from stim to response (remove noisy trials and trials with wrong answers)
            events = mne.find_events(raw)
            all_events = sub.get_all_events_times(id, events).dropna()
            new_raw = sub.segment_stimRt(raw, all_events, bad_trials)
            # interpolate bridged channels
            new_raw = mne.preprocessing.interpolate_bridged_electrodes(new_raw, bridged_channels['bridged_idx'], bad_limit=3) 
            # 4. compute ica 
            ica = mne.preprocessing.ICA(n_components = None, method= 'infomax', fit_params=dict(extended=True))
            ica.fit(new_raw, decim=2)
            ICA_path = os.path.join(workspace_root, 'Data', f'S{id}_ica_infomax_withoutnotch.fif')
            ica.save(ICA_path, overwrite=True)
        except Exception as e:
            print(f"Error processing subject {id}: {e}")
            continue
    return 

# submit with submitit one job 
from submitit import AutoExecutor
executor = AutoExecutor(folder="logdir")
ids = [601]
executor.update_parameters(cpus_per_task=4,  mem_gb=14,  timeout_min=600)
job = executor.submit(compute_ica, ids)


In [ ]:
## load data and funtions 
#########################################################################################
import sys
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict
# Add the workspace root to the Python path
workspace_root = os.path.abspath("..")
if workspace_root not in sys.path:
    sys.path.append(workspace_root)
from functions import preprocess, behaviorAnalysis
with open(os.path.join( workspace_root,'Data', 'bridged_channels_analysis.pkl'), "rb") as f:
    all_bridged_channels = pickle.load(f)

with open(os.path.join( workspace_root, 'Data', 'bad_channels_detected.pkl'), "rb") as f:
    all_bads = pickle.load(f)

ic